In [ ]:
import sys
sys.path.append("..")
from src.model.components.clip_text_autoencoder import CLIPAutoEncoder
from src.dataset.text import SimpleTokenizer
from pathlib import Path
import random
from src.utils.datatool import read_jsonlines
import torch

from lmdb_embeddings.reader import LmdbEmbeddingsReader

In [ ]:
descriptions = set()
samples = read_jsonlines("/data/vtt/meta/vtt_all.jsonl")
for sample in samples:
    for label in sample["annotation"]:
        descriptions.add(label["label"])
descriptions = list(sorted(list(descriptions)))
print(f"Total descriptions: {len(descriptions)}")
print(descriptions[:10])

In [ ]:
# vitb16_4
ckpt_path = Path("/log/exp/vtt/WikiHowDataModule.CLIPAutoEncoder.GenerationLoss.2023-01-14_00-11-16/checkpoints/last.ckpt")
# vitl14_4
ckpt_path = Path("/log/exp/vtt/WikiHowDataModule.CLIPAutoEncoder.GenerationLoss.2023-01-14_00-11-17/checkpoints/last.ckpt")
text_ae = CLIPAutoEncoder(from_decoder_ckpt=ckpt_path).cuda()
tokenizer = SimpleTokenizer()

In [ ]:
random.seed(0)
for i in range(10):
    text = random.choice(descriptions)
    embedding = text_ae.encode_raw(text)
    de_text = text_ae.decode_raw(embedding)
    print(f"Text: {text}")
    print(f"De-text: {de_text}")
    print()
